In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from helper import create_lags, create_date_features, convert_to_category, optuna_process


In [3]:
train = pd.read_parquet('data/train.snap.parquet')
train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1542,1.0,2015-04-19,11512,nan,nan,nan,nan,0,0,0,2.240234
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1543,0.0,2015-04-20,11512,nan,nan,nan,nan,0,0,0,2.240234
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1544,0.0,2015-04-21,11512,nan,nan,nan,nan,0,0,0,2.240234
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1545,0.0,2015-04-22,11512,nan,nan,nan,nan,0,0,0,2.240234
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1546,1.0,2015-04-23,11512,nan,nan,nan,nan,0,0,0,2.240234


In [4]:
train.shape

(12159132, 18)

In [5]:
train['d'].value_counts

<bound method IndexOpsMixin.value_counts of 0           d_1542
1           d_1543
2           d_1544
3           d_1545
4           d_1546
             ...  
12159127    d_1937
12159128    d_1938
12159129    d_1939
12159130    d_1940
12159131    d_1941
Name: d, Length: 12159132, dtype: category
Categories (1969, object): ['d_1', 'd_10', 'd_100', 'd_1000', ..., 'd_996', 'd_997', 'd_998', 'd_999']>

In [6]:
train['d'] = train['d'].astype(str)

# Create validation data
validation_data = train[(train['d'] >= 'd_1914') & (train['d'] <= 'd_1941')]

# Remove validation data from training data
train = train[~((train['d'] >= 'd_1914') & (train['d'] <= 'd_1941'))]

# Display the first few rows of the validation data
validation_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
372,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1914,2.0,2016-04-25,11613,nan,nan,nan,nan,0,0,0,2.240234
373,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1915,0.0,2016-04-26,11613,nan,nan,nan,nan,0,0,0,2.240234
374,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1916,0.0,2016-04-27,11613,nan,nan,nan,nan,0,0,0,2.240234
375,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1917,0.0,2016-04-28,11613,nan,nan,nan,nan,0,0,0,2.240234
376,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1918,0.0,2016-04-29,11613,nan,nan,nan,nan,0,0,0,2.240234


In [7]:
train = train.reset_index(drop=True)

In [8]:
validation_data = validation_data.reset_index(drop=True)

In [9]:
df_lag = train[['id', 'item_id', 'dept_id' , 'cat_id', 'store_id' , 'state_id' , 'd', 'y', 'date']]
df_lag_val = validation_data[['id', 'item_id', 'dept_id' , 'cat_id', 'store_id' , 'state_id' , 'd', 'y', 'date']]

In [10]:
df_lagged = create_lags(df_lag, 28)
df_lagged_val = create_lags(df_lag_val, 28)

In [11]:
df_lagged.drop(columns=['d','y'], inplace=True)
df_lagged.head()


,id,item_id,dept_id,cat_id,store_id,state_id,date,lag_1,lag_2,lag_3,...,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-19,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-20,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-21,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-22,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-23,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_lagged_val.drop(columns=['d','y'], inplace=True)
df_lagged_val.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date,lag_1,lag_2,lag_3,...,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-04-25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-04-26,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-04-27,0.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-04-28,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-04-29,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_date_features = create_date_features(train[['id', 'item_id', 'dept_id' , 'cat_id', 'store_id' , 'state_id' , 'd', 'date']], 'date')
df_date_features.drop(columns=['d'], inplace=True)
df_date_features.head()


,id,item_id,dept_id,cat_id,store_id,state_id,date,day,month,year,dayofweek,is_weekend
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-19,19,4,2015,6,True
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-20,20,4,2015,0,False
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-21,21,4,2015,1,False
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-22,22,4,2015,2,False
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-23,23,4,2015,3,False


In [14]:
df_date_features_val = create_date_features(validation_data[['id', 'item_id', 'dept_id' , 'cat_id', 'store_id' , 'state_id' , 'd', 'date']], 'date')
df_date_features_val.drop(columns=['d'], inplace=True)
df_date_features_val.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date,day,month,year,dayofweek,is_weekend
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-04-25,25,4,2016,0,False
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-04-26,26,4,2016,1,False
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-04-27,27,4,2016,2,False
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-04-28,28,4,2016,3,False
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2016-04-29,29,4,2016,4,False


In [15]:
# Merge dataframes
merged_df = train.merge(df_lagged, on=['id', 'item_id', 'dept_id' , 'cat_id', 'store_id' , 'state_id' , 'date'], how='inner')
merged_df = merged_df.merge(df_date_features, on=['id', 'item_id', 'dept_id' , 'cat_id', 'store_id' , 'state_id' , 'date'], how='inner')

merged_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,date,wm_yr_wk,...,lag_24,lag_25,lag_26,lag_27,lag_28,day,month,year,dayofweek,is_weekend
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1542,1.0,2015-04-19,11512,...,NaN,NaN,NaN,NaN,NaN,19,4,2015,6,True
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1543,0.0,2015-04-20,11512,...,NaN,NaN,NaN,NaN,NaN,20,4,2015,0,False
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1544,0.0,2015-04-21,11512,...,NaN,NaN,NaN,NaN,NaN,21,4,2015,1,False
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1545,0.0,2015-04-22,11512,...,NaN,NaN,NaN,NaN,NaN,22,4,2015,2,False
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1546,1.0,2015-04-23,11512,...,NaN,NaN,NaN,NaN,NaN,23,4,2015,3,False


In [16]:
merged_df.shape

(11305412, 51)

In [17]:
merged_df.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd', 'y',
       'date', 'wm_yr_wk', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_1',
       'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9',
       'lag_10', 'lag_11', 'lag_12', 'lag_13', 'lag_14', 'lag_15', 'lag_16',
       'lag_17', 'lag_18', 'lag_19', 'lag_20', 'lag_21', 'lag_22', 'lag_23',
       'lag_24', 'lag_25', 'lag_26', 'lag_27', 'lag_28', 'day', 'month',
       'year', 'dayofweek', 'is_weekend'],
      dtype='object')

In [18]:
merged_df.set_index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'date'], inplace=True)

In [19]:
merged_df.head()

d  \
id                          item_id     dept_id cat_id store_id state_id date                 
FOODS_1_001_CA_1_evaluation FOODS_1_001 FOODS_1 FOODS  CA_1     CA       2015-04-19  d_1542   
                                                                         2015-04-20  d_1543   
                                                                         2015-04-21  d_1544   
                                                                         2015-04-22  d_1545   
                                                                         2015-04-23  d_1546   

                                                                                       y  \
id                          item_id     dept_id cat_id store_id state_id date              
FOODS_1_001_CA_1_evaluation FOODS_1_001 FOODS_1 FOODS  CA_1     CA       2015-04-19  1.0   
                                                                         2015-04-20  0.0   
                                                                         2015-04-21  0.0   
                                                                         2015-04-22  0.0   
                                                                         2015-04-23  1.0   

                                                                                     wm_yr_wk  \
id                          item_id     dept_id cat_id store_id state_id date                   
FOODS_1_001_CA_1_evaluation FOODS_1_001 FOODS_1 FOODS  CA_1     CA       2015-04-19     11512   
                                                                         2015-04-20     11512   
                                                                         2015-04-21     11512   
                                                                         2015-04-22     11512   
                                                                         2015-04-23     11512   

                                                                                    event_name_1  \
id                          item_id     dept_id cat_id store_id state_id date                      
FOODS_1_001_CA_1_evaluation FOODS_1_001 FOODS_1 FOODS  CA_1     CA       2015-04-19          nan   
                                                                         2015-04-20          nan   
                                                                         2015-04-21          nan   
                                                                         2015-04-22          nan   
                                                                         2015-04-23          nan   

                                                                                    event_type_1  \
id                          item_id     dept_id cat_id store_id state_id date                      
FOODS_1_001_CA_1_evaluation FOODS_1_001 FOODS_1 FOODS  CA_1     CA       2015-04-19          nan   
                                                                         2015-04-20          nan   
                                                                         2015-04-21          nan   
                                                                         2015-04-22          nan   
                                                                         2015-04-23          nan   

                                                                                    event_name_2  \
id                          item_id     dept_id cat_id store_id state_id date                      
FOODS_1_001_CA_1_evaluation FOODS_1_001 FOODS_1 FOODS  CA_1     CA       2015-04-19          nan   
                                                                         2015-04-20          nan   
                                                                         2015-04-21          nan   
                                                                         2015-04-22          nan   
                                                                         2015-04-23          nan 

In [20]:
# Merge dataframes
merged_df_val = validation_data.merge(df_lagged_val, on=['id', 'item_id', 'dept_id' , 'cat_id', 'store_id' , 'state_id' , 'date'], how='inner')
merged_df_val = merged_df_val.merge(df_date_features_val, on=['id', 'item_id', 'dept_id' , 'cat_id', 'store_id' , 'state_id' , 'date'], how='inner')
merged_df_val.set_index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'date'], inplace=True)
merged_df_val.head()

d  \
id                          item_id     dept_id cat_id store_id state_id date                 
FOODS_1_001_CA_1_evaluation FOODS_1_001 FOODS_1 FOODS  CA_1     CA       2016-04-25  d_1914   
                                                                         2016-04-26  d_1915   
                                                                         2016-04-27  d_1916   
                                                                         2016-04-28  d_1917   
                                                                         2016-04-29  d_1918   

                                                                                       y  \
id                          item_id     dept_id cat_id store_id state_id date              
FOODS_1_001_CA_1_evaluation FOODS_1_001 FOODS_1 FOODS  CA_1     CA       2016-04-25  2.0   
                                                                         2016-04-26  0.0   
                                                                         2016-04-27  0.0   
                                                                         2016-04-28  0.0   
                                                                         2016-04-29  0.0   

                                                                                     wm_yr_wk  \
id                          item_id     dept_id cat_id store_id state_id date                   
FOODS_1_001_CA_1_evaluation FOODS_1_001 FOODS_1 FOODS  CA_1     CA       2016-04-25     11613   
                                                                         2016-04-26     11613   
                                                                         2016-04-27     11613   
                                                                         2016-04-28     11613   
                                                                         2016-04-29     11613   

                                                                                    event_name_1  \
id                          item_id     dept_id cat_id store_id state_id date                      
FOODS_1_001_CA_1_evaluation FOODS_1_001 FOODS_1 FOODS  CA_1     CA       2016-04-25          nan   
                                                                         2016-04-26          nan   
                                                                         2016-04-27          nan   
                                                                         2016-04-28          nan   
                                                                         2016-04-29          nan   

                                                                                    event_type_1  \
id                          item_id     dept_id cat_id store_id state_id date                      
FOODS_1_001_CA_1_evaluation FOODS_1_001 FOODS_1 FOODS  CA_1     CA       2016-04-25          nan   
                                                                         2016-04-26          nan   
                                                                         2016-04-27          nan   
                                                                         2016-04-28          nan   
                                                                         2016-04-29          nan   

                                                                                    event_name_2  \
id                          item_id     dept_id cat_id store_id state_id date                      
FOODS_1_001_CA_1_evaluation FOODS_1_001 FOODS_1 FOODS  CA_1     CA       2016-04-25          nan   
                                                                         2016-04-26          nan   
                                                                         2016-04-27          nan   
                                                                         2016-04-28          nan   
                                                                         2016-04-29          nan 

In [21]:
merged_df.columns.to_list()

['d',
 'y',
 'wm_yr_wk',
 'event_name_1',
 'event_type_1',
 'event_name_2',
 'event_type_2',
 'snap_CA',
 'snap_TX',
 'snap_WI',
 'sell_price',
 'lag_1',
 'lag_2',
 'lag_3',
 'lag_4',
 'lag_5',
 'lag_6',
 'lag_7',
 'lag_8',
 'lag_9',
 'lag_10',
 'lag_11',
 'lag_12',
 'lag_13',
 'lag_14',
 'lag_15',
 'lag_16',
 'lag_17',
 'lag_18',
 'lag_19',
 'lag_20',
 'lag_21',
 'lag_22',
 'lag_23',
 'lag_24',
 'lag_25',
 'lag_26',
 'lag_27',
 'lag_28',
 'day',
 'month',
 'year',
 'dayofweek',
 'is_weekend']

In [22]:
y_train = merged_df['y']
merged_df = merged_df.drop(columns=['y'])
merged_df = convert_to_category(merged_df, ['d', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_weekend'])

y_val = merged_df_val['y']
merged_df_val = merged_df_val.drop(columns=['y'])
merged_df_val = convert_to_category(merged_df_val, ['d', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_weekend'])



In [23]:
optuna_process(merged_df, merged_df_val, y_train, y_val)

[I 2025-03-04 00:01:10,139] A new study created in memory with name: no-name-b53c93f5-7db3-4dc8-aef8-6a9bfe166030
[I 2025-03-04 00:04:37,199] Trial 0 finished with value: 2.1558989948577105 and parameters: {'n_estimators': 375, 'learning_rate': 0.28570714885887566, 'num_leaves': 247}. Best is trial 0 with value: 2.1558989948577105.
[I 2025-03-04 00:08:51,390] Trial 1 finished with value: 2.1335441134813493 and parameters: {'n_estimators': 420, 'learning_rate': 0.055245405728306586, 'num_leaves': 131}. Best is trial 1 with value: 2.1335441134813493.
[I 2025-03-04 00:11:47,652] Trial 2 finished with value: 2.170381333844379 and parameters: {'n_estimators': 311, 'learning_rate': 0.2611910822747312, 'num_leaves': 220}. Best is trial 1 with value: 2.1335441134813493.
[I 2025-03-04 00:19:14,904] Trial 3 finished with value: 2.134557620405691 and parameters: {'n_estimators': 442, 'learning_rate': 0.01596950334578271, 'num_leaves': 294}. Best is trial 1 with value: 2.1335441134813493.
[I 2025-

Best trial:
  Value: 2.129439556789575
  Params: 
    n_estimators: 435
    learning_rate: 0.06167826688804347
    num_leaves: 299


LGBMRegressor(categorical_feature='auto', feature_pre_filter=False,
              force_col_wise=True, force_row_wise=False,
              learning_rate=0.06167826688804347, metric='rmse',
              n_estimators=435, n_jobs=-1, nan_mode='min', num_leaves=299,
              objective='regression', random_state=42, verbosity=-1)